In [30]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.vectorstores import Pinecone
from pinecone import Pinecone, ServerlessSpec
import pinecone
from langchain_pinecone import PineconeVectorStore
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate 
from langchain.llms import CTransformers
from transformers import pipeline, AutoModel
from langchain.llms.huggingface_pipeline import HuggingFacePipeline 

In [2]:
# Extract data from the pdf
def load_pdf(data):
    loader = DirectoryLoader(data,
                   glob="*.pdf",
                   loader_cls=PyPDFLoader)

    documents = loader.load()

    return documents

In [4]:
extracted_data = load_pdf("data/")

In [5]:
# extracted_data

In [6]:
# Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print("length of chunks: ", len(text_chunks))

length of chunks:  7020


In [8]:
# download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
embeddings = download_hugging_face_embeddings()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
# test for "hello world"
query_result = embeddings.embed_query("Hello world")
print("Query Length: ", len(query_result))

Query Length:  384


In [12]:
query_result

[-0.03447727486491203,
 0.031023185700178146,
 0.006734976079314947,
 0.026108963415026665,
 -0.039361998438835144,
 -0.16030248999595642,
 0.06692397594451904,
 -0.006441421806812286,
 -0.04745053872466087,
 0.014758860692381859,
 0.07087533921003342,
 0.05552748590707779,
 0.019193319603800774,
 -0.026251329109072685,
 -0.01010950282216072,
 -0.02694052830338478,
 0.02230740897357464,
 -0.022226683795452118,
 -0.1496926248073578,
 -0.017493069171905518,
 0.007676247972995043,
 0.054352328181266785,
 0.0032544571440666914,
 0.03172598406672478,
 -0.08462140709161758,
 -0.02940600924193859,
 0.05159571394324303,
 0.048124074935913086,
 -0.003314804984256625,
 -0.05827919393777847,
 0.04196930676698685,
 0.022210728377103806,
 0.12818880379199982,
 -0.022338958457112312,
 -0.011656232178211212,
 0.06292840093374252,
 -0.03287631273269653,
 -0.09122604876756668,
 -0.03117530420422554,
 0.05269956961274147,
 0.04703482985496521,
 -0.08420304954051971,
 -0.030056210234761238,
 -0.020744835

In [13]:
import os
PINECONE_API_KEY = "c92f6e26-04c2-4ac5-9fb7-54e572254d5d"
PINECONE_API_ENV = "gcp-starter"

os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
os.environ['PINECONE_API_ENV'] = PINECONE_API_ENV

In [18]:
text_field = [text.page_content for text in text_chunks]
index_name = "medical-chatbot"

# vectorstore = PineconeVectorStore.from_texts(
#     text_field, embeddings, index_name=index_name
# )

In [19]:
# lets query
docsearch = PineconeVectorStore.from_existing_index(index_name,embeddings)

query = "What are Allergies"

docs = docsearch.similarity_search(query, k=3)

print('Result', docs)

Result [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of drugs commonly ca

In [20]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you dont't know the answer, just say that you don't know. Don't try to make up an answer

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else:
Helpful answer: 
"""

In [31]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}

model = AutoModel.from_pretrained('model/llama-2-7b-chat.ggmlv3.q4_0.bin')

OSError: model/llama-2-7b-chat.ggmlv3.q4_0.bin is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [29]:
pipe = pipeline(model=model,
                 model_type="llama",
                 config={"max_new_tokens": 512,
                        "temperature": 0.8})

hf = HuggingFacePipeline(pipeline=pipe)

RuntimeError: Instantiating a pipeline without a task set raised an error: 404 Client Error. (Request ID: Root=1-65ed5c01-11e7c5402c7996c916c55dd5;3b29843d-6954-4288-b971-f122a2228cd6)

Repository Not Found for url: https://huggingface.co/api/models/llama-2-7b-chat.ggmlv3.q4_0.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.

In [44]:
pwd

'/scratch/Shaz/Personal_projects/Medical_chatbot'